In [9]:
import scipy.io
import numpy as np

In [10]:
mat_data = scipy.io.loadmat('./data_preprocessed_matlab/s02.mat') 
original_data = mat_data['data']
original_label = mat_data['labels']

# (40, 40, 8064) 40试验，40通道（前32为eeg），63s（前3秒没用） x 128Hz = 8064
print("original_data形状：", original_data.shape)
# (40, 4) 40试验，4个评价指标
print("original_label形状：", original_label.shape)

original_data形状： (40, 40, 8064)
original_label形状： (40, 4)


In [11]:
'''
    处理eeg，从(40, 40, 8064)变成(40, 32, 7680)
'''
# 前32通道为eeg，去除前3秒
sliced_data = original_data[:, :32, 384:] 
print("sliced_data形状：", sliced_data.shape) # (40, 32, 7680)

eeg_data = sliced_data

sliced_data形状： (40, 32, 7680)


In [12]:
'''
    处理labels，从(40, 4)变成(800,)
'''
valence = original_label[:,0]
arousal = original_label[:,1]
# HAHV--1, LAHV--2, LALV--3, HALV--4 完全按照象限来分
VA_labels = np.where((valence > 5) & (arousal > 5), 0,
          np.where((valence >= 5) & (arousal < 5), 1,
            np.where((valence < 5) & (arousal < 5), 2, 3)))
print("V:", valence)
print("A:", arousal)
print(VA_labels)

# 将数据切片成 3 秒一段
segment_size = 3 * 128
# 将 VA_labels 从 40 扩展到 800，VA_labels中的一个数复制20次
num_segments = sliced_data.shape[2] // segment_size # 7680/3/128 = 20
expanded_VA_labels = np.repeat(VA_labels, num_segments)
print(expanded_VA_labels.shape) # (800,)
labels = expanded_VA_labels 
# print(labels)

V: [9.   8.01 9.   6.05 5.04 5.   4.96 9.   9.   4.99 7.08 8.01 9.   8.94
 9.   6.   9.   9.   9.   8.97 1.   5.   1.   7.08 9.   9.   4.96 4.06
 2.99 2.01 4.06 1.   5.   4.87 1.99 5.04 1.   1.   1.   6.05]
A: [5.03 7.1  9.   1.   3.   4.94 1.99 9.   7.   1.   1.   6.06 4.99 9.
 8.01 5.05 3.   6.15 8.32 9.   1.   1.   2.97 9.   9.   8.04 5.01 4.97
 7.04 7.05 7.05 9.   8.03 1.   2.   1.   8.06 9.   1.   6.67]
[0 0 0 1 1 1 2 0 0 2 1 0 1 0 0 0 1 0 0 0 2 1 2 0 0 0 3 2 3 3 3 3 3 2 2 1 3
 3 2 0]
(800,)


'''
    保存数据
'''
np.save('./EEGData/s01_eeg.npy', eeg_data) # (40, 32, 7680)
np.save('./EEGData/s01_labels.npy', labels) # (800,)

In [13]:
import pandas as pd
import mne
from mne import io
from mne.datasets import sample
from mne.preprocessing import ICA
import os

sfreq = 128  # 采样率为128Hz
channels = 32
samples = 384
num_each_trial = 20
ch_names = ['Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 
            'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 
            'FC6', 'FC2', 'F4', 'F8', 'AF4', 'FP2', 'Fz', 'Cz']
ch_types = ['eeg'] * channels

data_list = []
eeg_data_segments = np.split(eeg_data, 40, axis=0) # (40, 32, 7680)
index = 0
for segment in eeg_data_segments:
    index = index + 1
    print("wzt index", index)
    segment_2d = segment.reshape(-1, channels).T
    print("wzt segment_2d", segment_2d.shape) # (32, 7680)
    # 创建MNE的Raw对象
    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
    raw = mne.io.RawArray(segment_2d, info=info)
    # 滤波
    raw.filter(l_freq=1.0, h_freq=50.0)
    # 创建ICA对象并拟合数据
    ica = ICA(n_components=channels, random_state=0, max_iter=1000)  # 调整参数
    ica.fit(raw)
    # 应用ICA滤波
    ica.exclude = []  # 将排除的独立成分列表设置为空
    ica.apply(raw)
    # 获取滤波后的数据
    data = raw.get_data().T # (7680, 32)

    # 将数据调整为每个trial的形状
    data = data[:num_each_trial * samples, :]
    # 重新将数据分成每个trial的形状
    data = data.reshape(num_each_trial, samples, channels)
    # 添加数据到列表中
    data_list.append(data)

# 将数据列表转换为numpy数组并按顺序连接
data_array = np.concatenate(data_list, axis=0) # (800, 384, 32)
# 交换位置
data_array = np.swapaxes(data_array, 1, 2) # (800, 32, 384)
print(data_array.shape) 
eeg_data = data_array

wzt index 1
wzt segment_2d (32, 7680)
Creating RawArray with float64 data, n_channels=32, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by number: 32 components


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 0.8s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
wzt index 2
wzt segment_2d (32, 7680)
Creating RawArray with float64 data, n_channels=32, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by numbe

/tmp/ipykernel_516038/1713448311.py:32: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (12) and smallest (7.8e-14) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 26
  ica.fit(raw)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 0.9s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
wzt index 3
wzt segment_2d (32, 7680)
Creating RawArray with float64 data, n_channels=32, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by numbe

...

/tmp/ipykernel_516038/1713448311.py:32: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (13) and smallest (1.1e-13) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 27
  ica.fit(raw)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 1.0s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
wzt index 40
wzt segment_2d (32, 7680)
Creating RawArray with float64 data, n_channels=32, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by numb

/tmp/ipykernel_516038/1713448311.py:32: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (12) and smallest (4.6e-14) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 26
  ica.fit(raw)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 0.9s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
(800, 32, 384)


/tmp/ipykernel_516038/1713448311.py:32: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (14) and smallest (9.7e-14) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 26
  ica.fit(raw)


In [14]:
'''
    保存数据
'''
np.save('./EEGData/s02_eeg.npy', eeg_data) # (800, 32, 384)
np.save('./EEGData/s02_labels.npy', labels) # (800,)